In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import pandas as pd

In [2]:
df = pd.read_excel('updated_dataset.xlsx')

In [3]:
df.fillna(0, inplace=True) 
df = df.drop(['use_metrics_na', 'org_field', 'agile_methods', 'sanitized_metrics', 'agile_methods_split'], axis=1)
df.head()

,Carimbo de data/hora,id_integer,role,years_exp,org_size,use_metrics_planning,use_metrics_review,use_metrics_weekly,use_metrics_daily,use_metrics_retro,metrics_category,agile_methods_filtered,matched_metrics,not_matched_metrics
0,17:34:49,1,Product manager,0 a 5,Microempresa,Métricas,0,Métricas,0,0,"Cronograma e progresso, Produto, Tecnologia, C...","['Scrum', 'Kanban']",['ideal team capacity'],[]
1,17:57:23,2,Desenvolvedor(a),0 a 5,Microempresa,Métricas,Métricas,0,0,0,"Cronograma e progresso, Produto, Cliente","['Scrum', 'Kanban']","['nps', 'burndown', 'outstanding bugs', 'throu...",[]
2,20:50:58,3,Product manager,0 a 5,Pequena empresa,Gerenciamento de riscos,Métricas,0,Gerenciamento de riscos,"Gerenciamento de riscos, Métricas","Cronograma e progresso, Pessoas, Produto",['Scrum'],"['burndown', 'velocity']","['okr', 'quantidade de solicitações de clientes']"
3,22:26:06,4,Team leader,6 a 9,Microempresa,"Gerenciamento de riscos, Métricas",Gerenciamento de riscos,Gerenciamento de riscos,Gerenciamento de riscos,Métricas,"Cronograma e progresso, Produto, Cliente","['Scrum', 'Kanban']",['throughput'],"['cac', 'roadmaps', 'conversão']"
4,19:07:11,5,Product manager,0 a 5,Pequena empresa,"Gerenciamento de riscos, Métricas",Métricas,0,0,Métricas,Cronograma e progresso,"['Scrum', 'Kanban']","['user story points', 'nps']",[]


In [4]:
df.to_excel('ready_to_classify.xlsx')

In [5]:
# Filtrar os dados e definir features e target
data_filtered = df.dropna(subset=['metrics_category'])
X = data_filtered[
    [
        'role',	
        'years_exp',	
        'org_size',	
        'use_metrics_planning',
        'use_metrics_review',
        'use_metrics_weekly',	
        'use_metrics_daily',	
        'use_metrics_retro',	
        'agile_methods_filtered']
    ].astype(str)


In [6]:
# Criar colunas binárias para cada categoria de métricas usando one-hot encoding
y = data_filtered['metrics_category'].str.get_dummies(sep=', ')

In [7]:
# Combinar todas as colunas de X em uma única coluna de texto
X_combined = X.apply(lambda row: ' '.join(row.values), axis=1)

In [8]:
# Inicializar o TfidfVectorizer
tfidf = TfidfVectorizer()

# Ajustar e transformar o conjunto de dados X em uma matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(X_combined)

In [9]:
# Dividir os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(tfidf_matrix, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [10]:
# Criar o classificador RandomForest como um classificador multi-label
classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

In [11]:
# Treinar o modelo com o conjunto de treinamento
classifier.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [12]:
# Fazer previsões no conjunto de validação
y_val_pred = classifier.predict(X_val)

In [13]:
# Gerar e imprimir o relatório de classificação para o conjunto de validação
print("Relatório de Classificação - Validação:")
report_val = classification_report(y_val, y_val_pred, target_names=y.columns)
print(report_val)

Relatório de Classificação - Validação:
                             precision    recall  f1-score   support

                    Cliente       0.31      0.40      0.35        10
     Cronograma e progresso       0.76      0.91      0.83        32
Nenhuma das respostas acima       0.00      0.00      0.00         0
                    Pessoas       0.31      0.25      0.28        20
                   Processo       0.55      0.52      0.53        23
                    Produto       0.64      1.00      0.78        25
                 Tecnologia       0.42      0.56      0.48        18

                  micro avg       0.56      0.66      0.61       128
                  macro avg       0.43      0.52      0.46       128
               weighted avg       0.55      0.66      0.59       128
                samples avg       0.59      0.71      0.60       128



/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this b

In [14]:
# Fazer previsões no conjunto de teste
y_test_pred = classifier.predict(X_test)

In [15]:
# Gerar e imprimir o relatório de classificação para o conjunto de teste
print("Relatório de Classificação - Teste:")
report_test = classification_report(y_test, y_test_pred, target_names=y.columns)
print(report_test)

Relatório de Classificação - Teste:
                             precision    recall  f1-score   support

                    Cliente       0.56      0.31      0.40        16
     Cronograma e progresso       0.80      0.90      0.85        31
Nenhuma das respostas acima       0.00      0.00      0.00         0
                    Pessoas       0.89      0.40      0.55        20
                   Processo       0.40      0.33      0.36        18
                    Produto       0.70      0.97      0.81        29
                 Tecnologia       0.41      0.50      0.45        18

                  micro avg       0.65      0.64      0.64       132
                  macro avg       0.54      0.49      0.49       132
               weighted avg       0.65      0.64      0.62       132
                samples avg       0.60      0.59      0.56       132



/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this b